In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [2]:
dataset = pd.read_csv(r'../../data/processed/charity_main_cleaned.csv')
ladataset = pd.read_excel(r'../../data/raw/Local_government_finance_data_download.xlsx',sheet_name='Spending power totals')
housing_net_supply_df = pd.read_excel('../../data/raw/Local-authority-housing-supply.xlsx', sheet_name='Net supply')
housing_stock_df = pd.read_excel('../../data/raw/Local-authority-housing-supply.xlsx', sheet_name='Housing stock')
population = pd.read_excel('../../data/raw/population_by_age.xlsx', sheet_name='Age bands')

C:\Users\User\AppData\Local\Temp\ipykernel_18540\925063635.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(r'../../data/processed/charity_main_cleaned.csv')


In [3]:
dataset.columns

Index(['registered_charity_number', 'charity_name', 'postcode',
       'charity_status', 'charity_type', 'date_of_registration',
       'date_of_removal', 'latest_income', 'charity_activities',
       'charity_has_land', 'companyNumber', 'local_authority', 'size_category',
       'classification_description', 'registration_year', 'removal_year',
       'registration_month', 'removal_month', 'registration_fy', 'removal_fy',
       'classification_other_charitable_activities',
       'classification_the_prevention_or_relief_of_poverty', 'Children_Youth',
       'Economic_Social Development', 'Economic_Social_Development',
       'Education_Research', 'Environment_Animals', 'Grants_Related',
       'Health_Disability', 'Housing_Infrastructure', 'Human_Rights_Advocacy',
       'Other_Miscellaneous', 'Overseas_Famine_Relief', 'Religion_Faith'],
      dtype='object')

In [ ]:
# Step 1: Calculate total 2024 housing stock per council (Local authority only)
stock_2024 = (
    housing_stock_df
    .groupby(['Authority_code', 'Authority_name'], as_index=False)['LA_number']
    .sum()
    .rename(columns={'LA_number': 'Stock_2024'})
)

# Step 2: Prepare Net Supply Data
net = housing_net_supply_df.copy()
net = net[net['Year'].astype(str).str.match(r'^\d{4}/\d{2}$')]  # Keep only year-formatted rows
net['Year'] = net['Year'].str[:4].astype(int)  # Convert '2015/16' → 2015

# Pivot to Local Authority x Year format
net_pivot = net.pivot(index='LA_code', columns='Year', values='Net_additions').fillna(0)
net_pivot = net_pivot[sorted(net_pivot.columns, reverse=True)]  # descending order

# Step 3: Merge with 2024 stock
stock = stock_2024.set_index('Authority_code')
net_pivot = net_pivot.reindex(stock.index)  # align index

# Step 4: Calculate total stock backwards from 2024
stock_by_year = pd.DataFrame(index=net_pivot.index)
stock_by_year[2024] = stock['Stock_2024']

for year in sorted(net_pivot.columns, reverse=True):
    if year < 2024:
        stock_by_year[year] = stock_by_year[year + 1] - net_pivot[year]

# Step 5: Merge back Authority name and reshape
stock_by_year = stock_by_year.merge(stock[['Authority_name']], left_index=True, right_index=True)
stock_by_year_reset = stock_by_year.reset_index().melt(id_vars=['Authority_code', 'Authority_name'], 
                                                       var_name='Year', value_name='Housing stock')

# Final pivot
stock_by_year_pivot = stock_by_year_reset.pivot(index=['Authority_code', 'Authority_name'], 
                                                 columns='Year', values='Housing stock').sort_index(axis=1)

# Reset index, rename, and set it again
stock_by_year_pivot = stock_by_year_pivot.reset_index().rename(columns={'Authority_code': 'ONS code'})
stock_by_year_pivot = stock_by_year_pivot.set_index(['ONS code', 'Authority_name'])


In [ ]:
# Filter: Remove NaN and keep only rows where 'Year' matches 'YYYY/YY'
ladataset_filtered = ladataset[
    ladataset['Year'].notna() & ladataset['Year'].str.match(r'^\d{4}/\d{2}$')
]
ladataset_filtered['Year'] = ladataset_filtered['Year'].str[:4].astype(int)

# Pivot
ladataset_pivot = ladataset_filtered.pivot_table(
    index=['ONS code', 'Local authority', 'Measure'],
    columns='Year',
    values='£ millions, cash terms',
    aggfunc='sum'
).reset_index()

# Group by 'con_code' only
population_summary = (
    population
    .groupby('con_code', as_index=False)
    .agg({'con_name': 'first', 'con_number': 'sum'})
    .rename(columns={'con_name': 'Local authority', 'con_number': 'Total Population'})
)

# Display the result
print(population_summary.head())


In [ ]:
# Filter for Core Spending Power (CSP)
csp = ladataset_pivot.copy()

# Keep only relevant years
csp = csp.set_index(['ONS code', 'Local authority']).drop(columns='Measure')

# Calculate % change from 2015
csp['financial_distress'] = (csp[2024] - csp[2015]) / csp[2015]

fd = csp[['financial_distress']].reset_index()

fd.columns.name = None
housing_stock_processed = housing_stock_df[housing_stock_df['Tenure'] == 'Local authority'][['Authority_code', 'LA_number']]
fd_new = fd.merge(housing_stock_processed, left_on='ONS code', right_on='Authority_code', how='inner')
fd_new = fd_new.drop(columns='Authority_code')
fd_new = fd_new.rename(columns={'LA_number': 'LA Housing stock 2024'})


In [ ]:
# Step 1: Normalise both variables
fd_new['financial_distress_norm'] = (
    (fd_new['financial_distress'] - fd_new['financial_distress'].min()) /
    (fd_new['financial_distress'].max() - fd_new['financial_distress'].min())
)

fd_new['LA Housing stock 2024_norm'] = (
    (fd_new['LA Housing stock 2024'] - fd_new['LA Housing stock 2024'].min()) /
    (fd_new['LA Housing stock 2024'].max() - fd_new['LA Housing stock 2024'].min())
)

# Step 2: Avoid division by zero and calculate treatment
fd_new['treatment'] = np.where(
    fd_new['LA Housing stock 2024_norm'] != 0,
    fd_new['financial_distress_norm'] / fd_new['LA Housing stock 2024_norm'],
    0
)
fd_new.rename(columns={'Local authority': 'local_authority'}, inplace=True)

In [ ]:
# Step 2: Filter valid rows
removed = dataset[dataset['removal_fy'].notnull() & dataset['local_authority'].notnull()].copy()
removed['removal_fy'] = removed['removal_fy'].astype(int)

# Step 3: Group by financial year and local authority
removed_by_fy_la = (
    removed
    .groupby(['local_authority', 'removal_fy'])
    .size()
    .unstack(fill_value=0)
    .sort_index(axis=1)
)

# Step 4: Keep only FY 2015–2024
fy_years = list(range(2015, 2025))
removed_by_fy_la = removed_by_fy_la[removed_by_fy_la.columns.intersection(fy_years)]

# Step 5: Total and sort
removed_by_fy_la['Total'] = removed_by_fy_la.sum(axis=1)
removed_by_fy_la = removed_by_fy_la.sort_values(by='Total', ascending=False)

# Step 6: Preview top 10
print("Removed Charities per Financial Year per Local Authority (FY 2015–2024):")
print(removed_by_fy_la.head(10))

In [ ]:
# Step 1: Define year range
years = list(range(2015, 2025))

# Step 2: Calculate percentage change across years
removed_pct_change = removed_by_fy_la[years].pct_change(axis=1) * 100

# Step 3: Round for readability
removed_pct_change = removed_pct_change.round(2)
removed_pct_change

In [ ]:
removed_long = (
    removed_pct_change
    .reset_index()
    .melt(id_vars='local_authority', var_name='Year', value_name='removed_rate')
)
removed_long['Year'] = removed_long['Year'].astype(int)

# Add treatment, post, and controls
panel_df = removed_long.merge(fd_new, on='local_authority', how='left')
panel_df['post'] = (panel_df['Year'] >= 2021).astype(int)
panel_df['treatment_post'] = panel_df['treatment'] * panel_df['post']

panel_df = panel_df[np.isfinite(panel_df['removed_rate'])]
panel_df['removed_rate'].dropna(inplace=True)
panel_df.drop(columns=['ONS code'], inplace=True)

In [ ]:
panel_df

In [ ]:
model = smf.ols('removed_rate ~ post + treatment + C(local_authority)+ treatment:post', data=panel_df).fit()
summary = model.summary()
print(summary.tables[0])  # Overview: R², F-stat, etc.
print(summary.tables[1].as_text())  # Full coefficient table
